# 1. Data

In [1]:
# using Pkg
# Pkg.add("DataFrames")
using CSV
using DataFrames
using ForwardDiff
using LinearAlgebra
using Optim

using Random
using Statistics

In [7]:
df = DataFrame()
df = CSV.read("ps3clean.csv", DataFrame)

,Data NuEer,bidder,house,date,lot,profit,Net Payment,rank,realisation in final auAtion
,Int64,Int64,String,Int64,String,Float64,Float64,Int64,Int64
1,1,1,A,35360,217,150.0,-150.0,1,5200
2,2,2,A,35360,217,150.0,150.0,2,5200
3,3,1,A,35360,218,0.0,0.0,1,1000
4,4,1,A,35360,220,0.0,0.0,1,1500
5,5,1,A,35360,222,0.0,0.0,1,4800
6,6,1,A,35360,350,0.0,0.0,1,2200
7,7,2,A,35360,350,0.0,0.0,2,2200
8,8,3,A,35360,350,0.0,0.0,3,2200
9,9,1,A,35360,351,0.0,0.0,1,1400


In [10]:
names(df)
# println(minimum(df[:,1]))
# println(maximum(df[:,1]))

21-element Array{String,1}:
 "Data NuEer"
 "bidder"
 "house"
 "date"
 "lot"
 "profit"
 "Net  Payment"
 "rank"
 "realisation in final auAtion"
 "bid"
 "EstDated MinDum"
 "EstDated MaxDum"
 "Aatalog PriAe"
 "Aatalog Du10y"
 "EstDate Min"
 "EstDate Max"
 "Grade Min "
 "Grade Max"
 "No Grade"
 "ExAlusively US"
 "No Value"